In [1]:
import pandas as pd
from pylab import rcParams
import seaborn as sb
import matplotlib.pyplot as plt

import sklearn
from sklearn.cluster import DBSCAN
from collections import Counter
import datetime

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [2]:
class DataFrameValidator:
    def __init__(self, df):
        self.df = df
    
    def validate(columns = [], minimumRowAmount=1):
        # TODO: validate columns
        # TODO: validate rows
        pass

        

In [3]:

to_milliseconds = lambda seconds : seconds * 60 * 1000

class BinaryDataAnalysis:
    min_samples_untill_its_a_cluster = 2
    
    def __init__(self, 
                 dataframe, 
                 eps=5, #minutes
                 cluster_degregation=2,
                 max_cluster_distance=7.5,   #minutes
                 weeks=5,
                 decay_strength=0.5,
                 cluster_threshold=25,
                 threshold_percentage=90):
        self.eps =                  eps
        self.cluster_degregation =  cluster_degregation
        self.max_cluster_distance = max_cluster_distance
        self.weeks =                weeks
        self.decay_strength =       decay_strength
        self.cluster_threshold =    cluster_threshold
        self.threshold_percentage = threshold_percentage
        
        starting_dataframe = self.clean_dataframe(dataframe)
        
        self.temp_clusters = self.get_clusters_recursive(starting_dataframe, eps)

    def clean_dataframe(self, df):
        d = defaultdict(LabelEncoder)
        df_fit = df.apply(lambda x: d[x.name].fit_transform(x))
        df_fit['state'] = df['state']
        df_fit['time'] = df['time']
        return df_fit

    
    def get_clusters_recursive(self, df, eps, iteration=0, cluster_arr=None):
        if cluster_arr is None:
            cluster_arr = []
            
        model = self.fit_model(df, eps)
        cluster_dict = self.get_clusters(df, model)
        
        for idx, df in cluster_dict['too_large'].items():
            cluster_arr + self.get_clusters_recursive(
                cluster_dict['too_large'][idx], 
                eps / self.cluster_degregation, 
                iteration + 1, 
                cluster_arr
            )
    
        for idx, df in cluster_dict['perfect_size'].items():
            cluster_arr.append(df)
        return cluster_arr
    
    
    def fit_model(self, df, eps):
        model = DBSCAN(
            eps=to_milliseconds(eps),
            min_samples=self.min_samples_untill_its_a_cluster
        ).fit(df)
        return model
    
    
    def get_clusters(self, df, model):
        
        df['cluster'] = model.labels_
        
        cluster_dict_too_large = {}
        cluster_dict_perfect_size = {}
        
        
        # Calculate amount of clusters
        cluster_data_count = Counter(model.labels_)
        if -1 in cluster_data_count:
            cluster_data_count.pop(-1) # don't count outliers as a cluster
        if (bool(cluster_data_count)):
            amount_of_clusters = max(cluster_data_count) + 1
        else:
            amount_of_clusters = 0;
        
        
        for idx in range(amount_of_clusters):
            cluster_df = df.loc[df['cluster'] == idx].drop(columns=['cluster'])
            
            first_time = cluster_df['time'].iloc[0]
            last_time = cluster_df['time'].iloc[cluster_df['time'].size - 1]
            diffrence_in_miliseconds = last_time - first_time
            if diffrence_in_miliseconds > to_milliseconds(self.max_cluster_distance):
                cluster_dict_too_large[idx] = cluster_df
            else:
                cluster_dict_perfect_size[idx] = cluster_df
        
        return {
            'too_large': cluster_dict_too_large,
            'perfect_size': cluster_dict_perfect_size
        }
        
    
    def calculate_groups(self):
        pass
    
        
        
        
        


In [4]:
address = '../datasets/staandelamp_realistic.json'
df_data = pd.read_json(address)
df_data = df_data.sort_values(by=['time'])
df_data = df_data[:400]
df_data.head()

,name,state,time
2,Staande_Lamp_3,0,1509489940655
6,Staande_Lamp_5,1,1509490011225
0,Staande_Lamp_1,1,1509491943009
1,Staande_Lamp_2,0,1509492221471
3,Staande_Lamp_3,1,1509492826941


In [5]:
import pprint
x = BinaryDataAnalysis(
    df_data
)
pprint.pprint(x.temp_clusters)

[    name  state           time
21     0      1  1509502111561
22     1      1  1509502163379,
     name  state           time
36     1      1  1509508178671
37     2      0  1509508285697
41     4      0  1509508398723,
      name  state           time
162     1      0  1509572450465
159     3      1  1509572567274
161     0      0  1509572662975,
      name  state           time
370     4      1  1509678410857
369     3      0  1509678519974,
      name  state           time
367     2      1  1509680676407
371     0      0  1509680699003,
    name  state           time
2     2      0  1509489940655
6     4      1  1509490011225,
    name  state           time
0     0      1  1509491943009
1     1      0  1509492221471,
    name  state           time
3     2      1  1509492826941
4     2      1  1509492860451
7     0      1  1509492904889
5     3      0  1509492910368
8     1      0  1509492983553,
     name  state           time
9      2      1  1509495776509
10     2      1  1509496